#### This cell imports necessary libraries in order for the code to work when using imported functions.

In [410]:
import os
import pandas as pd

import numpy
import pymysql
import mysql.connector
from sqlalchemy import create_engine

#### This cell gets the current directory, and reads a local csv data file.

In [411]:
data_dir = os.path.join(os.getcwd())
data_file = os.path.join(data_dir, 'dogs_cleaned.csv')

df = pd.read_csv(data_file, header=0, index_col=1)
#df.head()

#### This cell allows the user to interact with the data and choose which columns of information are relevant to them using the input method. The user enters the columns of data they want to see as list separated with commas (no spaces).

In [412]:
print("Please choose what information you would like to see about each dog breed.")
print("These are the column names:")
col_names = """
Dog Size, Dog Breed Group, Height, Weight, Life Span, Adaptability, Adapts Well To Apartment Living, 
Good For Novice Owners, Sensitivity Level, Tolerates Being Alone, Tolerates Cold Weather, Tolerates Hot Weather, 
All Around Friendliness, Affectionate With Family, Kid-Friendly, Dog Friendly, Friendly Towards Strangers,
Health And Grooming Needs, Amount Of Shedding, Drooling Potential, Easy To Groom, General Health, Potential For Weight Gain,
Size, Trainability, Easy To Train, Intelligence, Potential For Mouthiness, Prey Drive, Tendency To Bark Or Howl,
Wanderlust Potential, Physical Needs, Energy Level, Intensity, Exercise Needs, Potential For Playfulness
"""
print(col_names)
print("NOTE: Please enter the column names separated by commas with no spaces before and after the comma")
print("An example input would be: Dog Size,Height,Weight,Life Span")
select_cols = input("Please choose which columns of information you would like to see: ")
select_cols_list = select_cols.split(",")
print("Your selected columns are:", select_cols_list)

Please choose what information you would like to see about each dog breed.
These are the column names:

Dog Size, Dog Breed Group, Height, Weight, Life Span, Adaptability, Adapts Well To Apartment Living, 
Good For Novice Owners, Sensitivity Level, Tolerates Being Alone, Tolerates Cold Weather, Tolerates Hot Weather, 
All Around Friendliness, Affectionate With Family, Kid-Friendly, Dog Friendly, Friendly Towards Strangers,
Health And Grooming Needs, Amount Of Shedding, Drooling Potential, Easy To Groom, General Health, Potential For Weight Gain,
Size, Trainability, Easy To Train, Intelligence, Potential For Mouthiness, Prey Drive, Tendency To Bark Or Howl,
Wanderlust Potential, Physical Needs, Energy Level, Intensity, Exercise Needs, Potential For Playfulness

NOTE: Please enter the column names separated by commas with no spaces before and after the comma
An example input would be: Dog Size,Height,Weight,Life Span
Please choose which columns of information you would like to see: Heigh

#### This cell uses a try/catch to make sure the user entered column names that are in the data set with no spelling mistakes abiding by the syntax. The code in this cell catches these errors and tells the user what to do if this happens.

In [413]:
try:
    breed_id = numpy.arange(0, 391)
    df1 = df.get(['Breed Name'])
    df2 = df.get(select_cols_list)
    df = df1.join(df2, lsuffix='_caller', rsuffix='_other')
    df.insert(0, 'breed_id', breed_id, True)
except:
    print("You did not enter the column names correctly or abide by the syntax!")
    print("Please rerun the cell above and re-enter the information.")
    
    
# while retry == True:
#     select_cols = input("Please choose which columns of information you would like to see: ")
#     select_cols_list = select_cols.split(",")
#     column_input_check()


#### This cell uses input methods to ask the user to enter information regarding their local instance of MySQL server in an attempt to connect to their server.

In [414]:
# host_name = "localhost"
print("The following statements are regarding your local instance of MySQL server running on your device.")
host_name = input("Please enter the host name: ")
# host_ip = "127.0.0.1"
host_ip = input("Please enter the host ip address (default value is 127.0.0.1): ")
# port = "3306"
port = input("Please enter the port number: ")
# user_id = "root"
user_id = input("Please enter the user id: ")
# pwd = "$33cw@L4dB8s"
pwd = input("Please enter the password: ")
dst_dbname = "dog_breeds"

The following statements are regarding your local instance of MySQL server running on your device.
Please enter the host name: localhost
Please enter the host ip address (default value is 127.0.0.1): 127.0.0.1
Please enter the port number: 3306
Please enter the user id: root
Please enter the password: $33cw@L4dB8s


#### This cell uses a try/catch to check that it was able to successfully connect to a local instance of MySQL server on the user's laptop. If it couldn't connect successfully, it tells the user what the error may be and what to do to enter their information again.

In [415]:
try:
    conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)

    sqlEngine.execute(f"DROP DATABASE IF EXISTS `{dst_dbname}`;")
    sqlEngine.execute(f"CREATE DATABASE `{dst_dbname}`;")
    sqlEngine.execute(f"USE {dst_dbname};")
except:
    print("Unable to connect to your local instance of MySQL. Please make sure your SQL server is up and running and all information you have entered is correct! Re-run the above cell to re-enter your information.")

#### This cell defines the set_dataframe method that takes in a user id, password, host name, database name, dataframe name, primary key column, and database operation to connect to the local instance of MySQL and perform the indicated operation.

In [416]:
def set_dataframe(user_id, pwd, host_name, db_name, df, table_name, pk_column, db_operation):
    conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}/{db_name}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    
#     if db_operation == "insert":
#         #df.to_sql(table_name, con=connection, index=False, if_exists='replace')
#         sqlEngine.execute(f"ALTER TABLE {table_name} ADD PRIMARY KEY ('{pk_column}');")
#          #sqlEngine.execute(f"ALTER TABLE {table_name} ADD PRIMARY KEY ({pk_column});")
#         #print(f"ALTER TABLE {table_name} ADD PRIMARY KEY ('{pk_column}');")
    if db_operation == "insert":
        df.to_sql(table_name, con=connection, index=False, if_exists='replace')
        sqlEngine.execute(f"ALTER TABLE {table_name} ADD PRIMARY KEY ({pk_column});")
            
    elif db_operation == "update":
        df.to_sql(table_name, con=connection, index=False, if_exists='append')
    
    connection.close()

#### This cell contains code that sets the database operation as insert and defines a list of tables to be inserted into MySQL.

In [417]:
db_operation = "insert"

tables = [('dog_info', df, 'breed_id')]

#### This cell contains code to insert the dataframe into MySQL as a table with the specified primary key. It is enclosed in a try/catch to make sure there were no errors in the previous step and if it successfully inserts the dataframe into MySQL, it will provide the user with a sample query to view the table.

In [418]:
try:
    for table_name, dataframe, primary_key in tables:
        set_dataframe(user_id, pwd, host_name, dst_dbname, dataframe, table_name, primary_key, db_operation)
    print("""Hooray! A dataframe has successfully been inserted into your local instance of MySQL!
    Type the following query to view the table: SELECT * FROM dog_breeds.dog_info
    It should contain all the information you specified for each dog breed.
    Note: You may need to refresh your schemas to see the dog_breeds schema.""")
except:
    print("Error inserting dataframe into your local instance of MySQL! Make sure there are no other errors in the previous steps.")

Hooray! A dataframe has successfully been inserted into your local instance of MySQL!
    Type the following query to view the table: SELECT * FROM dog_breeds.dog_info
    It should contain all the information you specified for each dog breed.


#### This cell uses the shape method to print the number of rows and columns in the table.

In [419]:
# print(df.shape)
print("There are", df.shape[0], "rows(records) of data")
print("There are", df.shape[1], "columns of data")

There are 391 rows(records) of data
There are 8 columns of data
